<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import os
import pandas as pd
from torchvision.io import read_image

In [75]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [76]:
num_classes = 11
num_epochs = 2
batch_size = 2
learning_rate = 0.001
input_size = 64
sequence_length = 64
hidden_size = 128
num_layers = 2

In [79]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path) / 255
        label = self.img_labels.iloc[idx, 1] / 10
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

dataset = CustomImageDataset('data.csv', 'trainingData')
train_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

In [80]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.rnn(x, h0)  
        out = out[:, -1, :]
        out = self.fc(out)
        return out


In [81]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [82]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [83]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        labels = labels.type(torch.LongTensor)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [10/126], Loss: 0.0248
Epoch [1/2], Step [20/126], Loss: 3.6469
Epoch [1/2], Step [30/126], Loss: 2.2203
Epoch [1/2], Step [40/126], Loss: 1.3762
Epoch [1/2], Step [50/126], Loss: 3.7078
Epoch [1/2], Step [60/126], Loss: 2.8431
Epoch [1/2], Step [70/126], Loss: 4.1841
Epoch [1/2], Step [80/126], Loss: 0.3217
Epoch [1/2], Step [90/126], Loss: 0.6142
Epoch [1/2], Step [100/126], Loss: 2.0337
Epoch [1/2], Step [110/126], Loss: 2.7226
Epoch [1/2], Step [120/126], Loss: 3.8848
Epoch [2/2], Step [10/126], Loss: 2.2314
Epoch [2/2], Step [20/126], Loss: 3.1222
Epoch [2/2], Step [30/126], Loss: 3.6302
Epoch [2/2], Step [40/126], Loss: 3.8602
Epoch [2/2], Step [50/126], Loss: 4.9388
Epoch [2/2], Step [60/126], Loss: 2.5483
Epoch [2/2], Step [70/126], Loss: 3.4451
Epoch [2/2], Step [80/126], Loss: 0.4617
Epoch [2/2], Step [90/126], Loss: 2.3113
Epoch [2/2], Step [100/126], Loss: 2.2745
Epoch [2/2], Step [110/126], Loss: 3.8612
Epoch [2/2], Step [120/126], Loss: 4.1170
